# Experiment

### Vorbereitung

In [8]:
import pandas as pd
import numpy as np
import time

import experiment
from imputer import *

In [9]:
t0 = time.time()
results = []
base = experiment.reset_base()

In [10]:
functions =  {'Backfill':impute_backfill, 'Overall Mean':impute_overall_means, 
              'Yearly Mean':impute_yearly_means, 'Yearly Mean per Region':impute_yearly_means_per_region, 
              'Interpolate 3':interpolate3, 'Interpolate all':interpolate_all, 
              'ICE 1':iterative_imputer1, 'ICE 2':iterative_imputer2, 
              'ICE 3':iterative_imputer3, 'MICE':mice_imputer, 
              'KNN 1':knn_imputer1, 'KNN 2':knn_imputer2
             }

In [11]:
def run_all(frac):
    
    #getting n different training sets
    cords_list =[]
    n = 7
    for i in np.arange(1,n+1):
        cords_list.append(experiment.get_cords(frac, i))
    
    #setting up two dictionaries to save detailed and averaged results
    detailed_results = {}   
    results = {}
         
    #itterating over dictionary of all imputers
    for name, func in functions.items():
        func_runs = []
        print(name)
        
        #running each imputer n times on different training sets
        for count, cords in enumerate(cords_list):            
            print(f'run: {count+1}/{n}')
            
            #running current imputer and timeing runtime
            t0 = time.time()
            df= func(experiment.reset_train(cords)) 
            t1 = time.time()
            t = t1-t0
            
            #evaluating run
            func_runs.append(experiment.evaluate(df, t, cords))
        
        #print evaluation result of all runs of current imputer
        print(func_runs)
        print ('')
        
        #save detailed and averaged results
        detailed_results[name] = func_runs
        results[name] = np.mean(np.array(func_runs), axis=0)
        
    return results, detailed_results


### Durchlauf mit 3% fehlenden Werten

In [12]:
r1, r1_detail = run_all(0.03)

Testdaten mit 5.0% fehlenden Werten (absolut: 8515)
Testdaten mit 5.0% fehlenden Werten (absolut: 8515)
Testdaten mit 5.0% fehlenden Werten (absolut: 8515)
Testdaten mit 5.0% fehlenden Werten (absolut: 8515)
Testdaten mit 5.0% fehlenden Werten (absolut: 8515)
Testdaten mit 5.0% fehlenden Werten (absolut: 8515)
Testdaten mit 5.0% fehlenden Werten (absolut: 8515)
Backfill
run: 1/7
r2: -0.2630386067986965, rmse: 3.1264580984032304, t: 0.8395938873291016

run: 2/7
r2: -2.77533240911633, rmse: 1.1896157293266358, t: 0.8130342960357666

run: 3/7
r2: -1.688280205196218, rmse: 1.0226797653748734, t: 0.8214783668518066

run: 4/7
r2: -0.6672076075877178, rmse: 2.085863590967128, t: 0.8130960464477539

run: 5/7
r2: -0.07720522781720884, rmse: 3.533882957257935, t: 0.8215644359588623

run: 6/7
r2: -5.6191599013569355, rmse: 1.7468428288779958, t: 0.8175332546234131

run: 7/7
r2: -3.5433278053623427, rmse: 1.259265501149907, t: 0.8173270225524902

[[-0.2630386067986965, 3.1264580984032304, 32924, 0

C:\Users\mbfbr\Anaconda3\envs\other\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


r2: 0.9942158015767555, rmse: 0.20756410920051788, t: 10.276995420455933

run: 2/7


C:\Users\mbfbr\Anaconda3\envs\other\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


r2: 0.7827329900210515, rmse: 0.28204132134606086, t: 10.111709833145142

run: 3/7


...

C:\Users\mbfbr\Anaconda3\envs\other\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


r2: 0.8455905778080721, rmse: 0.2627399535604548, t: 77.50361061096191

run: 7/7
[IterativeImputer] Completing matrix with shape (4898, 165)
[IterativeImputer] Change: 3.0226881281120496e+16, scaled tolerance: 35084726045503.402 
[IterativeImputer] Change: 2383647377232018.0, scaled tolerance: 35084726045503.402 
[IterativeImputer] Change: 3231930426572498.0, scaled tolerance: 35084726045503.402 
[IterativeImputer] Change: 2638754076774955.0, scaled tolerance: 35084726045503.402 
[IterativeImputer] Change: 1041624632446648.2, scaled tolerance: 35084726045503.402 
[IterativeImputer] Change: 3158692042071690.0, scaled tolerance: 35084726045503.402 
[IterativeImputer] Change: 1161951696019282.5, scaled tolerance: 35084726045503.402 
[IterativeImputer] Change: 3416853970920924.0, scaled tolerance: 35084726045503.402 
[IterativeImputer] Change: 1360365335709377.0, scaled tolerance: 35084726045503.402 
[IterativeImputer] Change: 471121006398974.56, scaled tolerance: 35084726045503.402 


C:\Users\mbfbr\Anaconda3\envs\other\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


r2: 0.9244305270561566, rmse: 0.1597754590954746, t: 74.80313658714294

[[0.6608870903592907, 1.5892888267905176, 0, 83.12750625610352], [0.8401332984275719, 0.24193285700689618, 0, 83.12379908561707], [0.44034797517710433, 0.4595094866419774, 0, 74.11652898788452], [0.7923310971040671, 0.7245819483282079, 0, 78.15291094779968], [0.7486558176825678, 1.679236113665864, 0, 80.95230627059937], [0.8455905778080721, 0.2627399535604548, 0, 77.50361061096191], [0.9244305270561566, 0.1597754590954746, 0, 74.80313658714294]]

MICE
run: 1/7
Imputation round 0
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 14.34
[IterativeImputer] Ending imputation round 2/10, elapsed time 29.27


KeyboardInterrupt: 

In [8]:
r1 = pd.DataFrame(r1).T
r1.columns = ['r2', 'rmse', 'still_missing', 't']
r1

,0,1,2,3
Backfill,-2.090507,1.994944,32841.000000,0.883142
Overall Mean,-0.375690,1.520351,0.000000,1.148196
Yearly Mean,0.003883,1.443118,52298.000000,0.975204
Yearly Mean per Region,-0.037073,1.459472,58352.714286,1.096290
Interpolate 3,-0.821449,1.713417,32871.428571,0.954637
Interpolate all,-0.821336,1.685872,58.000000,0.932401
ICE 1,0.958789,0.173133,0.000000,9.967214
ICE 2,0.936942,0.354860,0.000000,24.333753
ICE 3,0.744604,0.731466,0.000000,85.888058
MICE,0.961106,0.190552,0.000000,139.684988


In [9]:
r1_detail = pd.DataFrame(pd.DataFrame(r1_detail).T.stack().to_dict()).T
r1_detail.columns = ['r2', 'rmse', 'still_missing', 't']
r1_detail

r2      rmse  still_missing          t
Backfill 0 -0.263039  3.126458        32924.0   0.891556
         1 -2.775332  1.189616        32825.0   0.886016
         2 -1.688280  1.022680        32824.0   0.877579
         3 -0.667208  2.085864        32841.0   0.872048
         4 -0.077205  3.533883        32828.0   0.874590
...              ...       ...            ...        ...
KNN 2    2  0.018198  0.608622            0.0  18.870671
         3  0.039752  1.558094            0.0  18.975445
         4  0.365186  2.668704            0.0  18.601710
         5 -0.009893  0.671935            0.0  18.870543
         6  0.650886  0.343416            0.0  18.801567

[84 rows x 4 columns]

### Durchlauf mit 5% fehlenden Werten

In [10]:
r2 , r2_detail =  run_all(0.05)

Testdaten mit 7.5% fehlenden Werten (absolut: 12773)
Testdaten mit 7.5% fehlenden Werten (absolut: 12773)
Testdaten mit 7.5% fehlenden Werten (absolut: 12773)
Testdaten mit 7.5% fehlenden Werten (absolut: 12773)
Testdaten mit 7.5% fehlenden Werten (absolut: 12773)
Testdaten mit 7.5% fehlenden Werten (absolut: 12773)
Testdaten mit 7.5% fehlenden Werten (absolut: 12773)
Backfill
run: 1/7
r2: -0.7622360204251646, rmse: 1.9101575370444133, t: 1.252185583114624

run: 2/7
r2: -0.13326755926364164, rmse: 1.5732663625326984, t: 1.2441728115081787

run: 3/7
r2: -0.15471382925960686, rmse: 2.34729184379117, t: 1.245006799697876

run: 4/7
r2: -0.38762500614250417, rmse: 2.1233606768359277, t: 1.2373647689819336

run: 5/7
r2: -0.12638032635757868, rmse: 1.8674634798834449, t: 1.2455017566680908

run: 6/7
r2: -0.1353819730423571, rmse: 1.8854985255726455, t: 1.239548683166504

run: 7/7
r2: -1.259193916505637, rmse: 1.778120493027196, t: 1.2486271858215332

[[-0.7622360204251646, 1.9101575370444133,

C:\Users\schwa\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


r2: 0.9803957096839419, rmse: 0.19838139998923512, t: 10.240909337997437

run: 2/7


C:\Users\schwa\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


r2: 0.9755494244776735, rmse: 0.22712199683626336, t: 10.305448293685913

run: 3/7


...

[IterativeImputer] Ending imputation round 8/10, elapsed time 10.06
[IterativeImputer] Ending imputation round 9/10, elapsed time 11.29
[IterativeImputer] Ending imputation round 10/10, elapsed time 12.48
Imputation round 7
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.24
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.45
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.69
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.91
[IterativeImputer] Ending imputation round 5/10, elapsed time 6.18
[IterativeImputer] Ending imputation round 6/10, elapsed time 7.41
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.67
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.90
[IterativeImputer] Ending imputation round 9/10, elapsed time 11.15
[IterativeImputer] Ending imputation round 10/10, elapsed time 12.38
Imputation round 8
[IterativeImputer] Compl

[IterativeImputer] Ending imputation round 6/10, elapsed time 6.94
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.08
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.25
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.39
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.55
Imputation round 6
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.16
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.32
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.44
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.63
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.78
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.95
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.10
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.28
[IterativeImputer] Ending imputation round 9/10

[IterativeImputer] Ending imputation round 4/10, elapsed time 4.61
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.75
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.93
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.05
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.22
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.36
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.52
Imputation round 5
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.14
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.30
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.44
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.59
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.73
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.90
[IterativeImputer] Ending imputation round 7/10

[IterativeImputer] Ending imputation round 2/10, elapsed time 2.32
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.48
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.62
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.79
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.90
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.06
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.19
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.37
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.51
Imputation round 4
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.16
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.31
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.48
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.62
[IterativeImputer] Ending imputation round 5/10

r2: -1.602264991548012, rmse: 2.2856048436310656, t: 19.485417366027832

run: 2/7
r2: 0.3021798150724092, rmse: 1.2133509054211025, t: 19.627933025360107

run: 3/7
r2: 0.46955096762140014, rmse: 1.5636010429853207, t: 19.485605001449585

run: 4/7
r2: 0.3733797406161049, rmse: 1.4021077287819956, t: 19.581395387649536

run: 5/7
r2: 0.12532994575125067, rmse: 1.619596599110858, t: 19.49038553237915

run: 6/7
r2: 0.08000077527047134, rmse: 1.6680486031875468, t: 19.675562143325806

run: 7/7
r2: 0.2418860703417669, rmse: 1.0132411753116484, t: 19.355597972869873

[[-1.602264991548012, 2.2856048436310656, 0, 19.485417366027832], [0.3021798150724092, 1.2133509054211025, 0, 19.627933025360107], [0.46955096762140014, 1.5636010429853207, 0, 19.485605001449585], [0.3733797406161049, 1.4021077287819956, 0, 19.581395387649536], [0.12532994575125067, 1.619596599110858, 0, 19.49038553237915], [0.08000077527047134, 1.6680486031875468, 0, 19.675562143325806], [0.2418860703417669, 1.0132411753116484, 0

In [11]:
r2 = pd.DataFrame(r2).T
r2.columns = ['r2', 'rmse', 'still_missing', 't']
r2

,0,1,2,3
Backfill,-0.422686,1.926451,33270.142857,1.244630
Overall Mean,-0.073224,1.679756,0.000000,1.507036
Yearly Mean,0.015852,1.619293,52298.000000,1.327044
Yearly Mean per Region,0.053835,1.605266,58435.571429,1.446808
Interpolate 3,-0.248698,1.837525,33301.142857,1.319927
Interpolate all,-0.249796,1.808529,59.142857,1.291759
ICE 1,0.988829,0.157992,0.000000,10.254815
ICE 2,0.956405,0.306609,0.000000,24.391101
ICE 3,0.824507,0.660810,0.000000,91.371202
MICE,0.991169,0.143301,0.000000,148.374906


In [12]:
r2_detail = pd.DataFrame(pd.DataFrame(r2_detail).T.stack().to_dict()).T
r2_detail.columns = ['r2', 'rmse', 'still_missing', 't']
r2_detail

r2      rmse  still_missing          t
Backfill 0 -0.762236  1.910158        33252.0   1.252186
         1 -0.133268  1.573266        33280.0   1.244173
         2 -0.154714  2.347292        33292.0   1.245007
         3 -0.387625  2.123361        33312.0   1.237365
         4 -0.126380  1.867463        33191.0   1.245502
...              ...       ...            ...        ...
KNN 2    2  0.469551  1.563601            0.0  19.485605
         3  0.373380  1.402108            0.0  19.581395
         4  0.125330  1.619597            0.0  19.490386
         5  0.080001  1.668049            0.0  19.675562
         6  0.241886  1.013241            0.0  19.355598

[84 rows x 4 columns]

### Durchlauf mit 10% fehlenden Werten

In [13]:
r3, r3_detail = run_all(0.1)

Testdaten mit 10.0% fehlenden Werten (absolut: 17030)
Testdaten mit 10.0% fehlenden Werten (absolut: 17030)
Testdaten mit 10.0% fehlenden Werten (absolut: 17030)
Testdaten mit 10.0% fehlenden Werten (absolut: 17030)
Testdaten mit 10.0% fehlenden Werten (absolut: 17030)
Testdaten mit 10.0% fehlenden Werten (absolut: 17030)
Testdaten mit 10.0% fehlenden Werten (absolut: 17030)
Backfill
run: 1/7
r2: -3.964219549140708, rmse: 1.1521878884113992, t: 1.6051125526428223

run: 2/7
r2: -1.8373707535864638, rmse: 1.6792466658151293, t: 1.5990629196166992

run: 3/7
r2: -1.9081389310564236, rmse: 1.5239926108614468, t: 1.5961840152740479

run: 4/7
r2: -0.8977876217879388, rmse: 1.5383638773729555, t: 1.6145422458648682

run: 5/7
r2: -1.1760918948068961, rmse: 1.4134849682042556, t: 1.6395134925842285

run: 6/7
r2: -6.042109199979188, rmse: 1.7577308194911425, t: 1.5969390869140625

run: 7/7
r2: -0.965017536607846, rmse: 1.5948064365763108, t: 1.6045513153076172

[[-3.964219549140708, 1.15218788841

C:\Users\schwa\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


r2: 0.9370202334659707, rmse: 0.12777336992410396, t: 10.586702346801758

run: 2/7


C:\Users\schwa\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


r2: 0.9521427024799756, rmse: 0.2140660895293748, t: 10.648990392684937

run: 3/7


...

[IterativeImputer] Ending imputation round 8/10, elapsed time 9.26
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.41
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.58
Imputation round 7
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.14
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.29
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.44
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.60
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.73
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.91
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.06
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.22
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.36
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.53
Imputation round 8
[IterativeImputer] Comple

[IterativeImputer] Ending imputation round 6/10, elapsed time 7.08
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.24
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.38
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.55
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.69
Imputation round 6
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.15
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.28
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.46
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.62
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.78
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.92
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.10
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.26
[IterativeImputer] Ending imputation round 9/10

[IterativeImputer] Ending imputation round 4/10, elapsed time 4.63
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.77
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.92
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.05
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.19
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.35
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.49
Imputation round 5
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.15
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.32
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.45
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.60
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.75
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.90
[IterativeImputer] Ending imputation round 7/10

[IterativeImputer] Ending imputation round 2/10, elapsed time 2.29
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.42
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.60
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.74
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.89
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.04
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.22
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.36
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.59
Imputation round 4
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.14
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.31
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.46
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.62
[IterativeImputer] Ending imputation round 5/10

r2: 0.2955192171799377, rmse: 0.42734084191608557, t: 20.19080090522766

run: 2/7
r2: 0.19846267489319835, rmse: 0.8760636601263931, t: 20.315165996551514

run: 3/7
r2: -0.010296713250978806, rmse: 0.8827533199698283, t: 20.263155221939087

run: 4/7
r2: 0.17378061493706398, rmse: 0.9979210178375953, t: 20.271790027618408

run: 5/7
r2: 0.17744053064572496, rmse: 0.8546920863246453, t: 20.35792851448059

run: 6/7
r2: 0.19856669014701844, rmse: 0.5824550843672418, t: 20.374468088150024

run: 7/7
r2: 0.16675994424132778, rmse: 1.0226470021977652, t: 20.222707509994507

[[0.2955192171799377, 0.42734084191608557, 0, 20.19080090522766], [0.19846267489319835, 0.8760636601263931, 0, 20.315165996551514], [-0.010296713250978806, 0.8827533199698283, 0, 20.263155221939087], [0.17378061493706398, 0.9979210178375953, 0, 20.271790027618408], [0.17744053064572496, 0.8546920863246453, 0, 20.35792851448059], [0.19856669014701844, 0.5824550843672418, 0, 20.374468088150024], [0.16675994424132778, 1.0226470

In [14]:
r3 = pd.DataFrame(r3).T
r3.columns = ['r2', 'rmse', 'still_missing', 't']
r3

,0,1,2,3
Backfill,-2.398676,1.522830,33689.285714,1.607987
Overall Mean,-0.395046,1.014262,0.000000,1.862622
Yearly Mean,0.006562,0.877928,52320.571429,1.690804
Yearly Mean per Region,-0.047670,0.899047,58597.428571,1.799179
Interpolate 3,-0.891135,1.183015,33720.000000,1.684518
Interpolate all,-0.892295,1.163724,58.428571,1.654335
ICE 1,0.959140,0.166005,0.000000,10.599425
ICE 2,0.958234,0.169590,0.000000,24.853248
ICE 3,0.759685,0.427929,0.000000,86.695862
MICE,0.969067,0.143948,0.000000,140.654344


In [15]:
r3_detail = pd.DataFrame(pd.DataFrame(r3_detail).T.stack().to_dict()).T
r3_detail.columns = ['r2', 'rmse', 'still_missing', 't']
r3_detail

r2      rmse  still_missing          t
Backfill 0 -3.964220  1.152188        33619.0   1.605113
         1 -1.837371  1.679247        33786.0   1.599063
         2 -1.908139  1.523993        33654.0   1.596184
         3 -0.897788  1.538364        33643.0   1.614542
         4 -1.176092  1.413485        33679.0   1.639513
...              ...       ...            ...        ...
KNN 2    2 -0.010297  0.882753            0.0  20.263155
         3  0.173781  0.997921            0.0  20.271790
         4  0.177441  0.854692            0.0  20.357929
         5  0.198567  0.582455            0.0  20.374468
         6  0.166760  1.022647            0.0  20.222708

[84 rows x 4 columns]

### Imputationverfahren auf Originaldatensatz anwenden

In [17]:
for name, func in functions.items():
    print(name)

    t0 = time.time()
    df= func(experiment.reset_base()) 
    t1 = time.time()
    t = t1-t0

    print(f'{round(t, 2)}s vergangen')
    print('')
    df.to_csv(f'additional_data/imputed_sets/{name}.csv')

Backfill
0.18s vergangen

Overall Mean
0.48s vergangen

Yearly Mean
0.31s vergangen

Yearly Mean per Region
0.39s vergangen

Interpolate 3
0.24s vergangen

Interpolate all
0.21s vergangen

ICE 1


KeyboardInterrupt: 

### Ergebnisse abspeichern

In [16]:
t1 = time.time()
t = t1-t0
print(f'comlete runtime: {t/60}min')

comlete runtime: 164.17220553557078min


In [17]:
sets = {'r1':r1, 'r1_detail':r1_detail,
       'r2':r2, 'r2_detail':r2_detail,
       'r3':r3, 'r3_detail':r3_detail,}
for name, df in sets.items():
    df.to_csv(f'additional_data/results/{name}.csv')